# CEE6501 — Coding Assignment, Week 3

**Assigned:** 01/30/2026 (Week 3)  
**Due:** 02/09/2026

**Canvas Submission Link:**  <https://gatech.instructure.com/courses/517856/assignments/2320756>

---

## Logistics

### 💻 Assignment Format

This is a **coding assignment**.

- Complete the assignment by **executing and completing all tasks in the notebook cells below**
- The notebook should be run and completed in **Google Colab**
- Your submission **must be a link to a functioning Google Colab notebook**

You may use any local tools (VS Code, JupyterLab, etc.) while working, but the **final submitted work must run correctly in Colab**.


### 📤 Submission Instructions

- Submit **one link** to your Google Colab notebook on Canvas
- Ensure that:
  - All cells run **top-to-bottom without errors**
  - All required outputs are visible
  - The notebook reflects your final answers

### ✅ Checklist Before Submitting

- [ ] All notebook cells completed
- [ ] Code runs without errors from a fresh runtime
- [ ] Outputs and plots are clearly visible
- [ ] Colab link opens and runs correctly
- [ ] Correct notebook submitted on Canvas

### 🤝 Collaboration / AI tools
You may discuss concepts with classmates and you may use AI tools to help you learn,
but **your submitted code must be written by you and you must understand it**.
If you used outside help, add a short note in the final reflection cell.

---

## --- Google Colab environment setup ---

The cell below only needs to run when the notebook is opened in Google Colab.

This code will not affect code execution locally in VS-code + conda environment.

Google Colab starts each session with its own **preloaded versions** of common Python (currently 3.12.12) and Python packages (NumPy, SciPy, etc.).  
If we install different package versions once loaded, Python cannot switch to them while it is already running.

### What will happen
When you run the setup cell below in Google Colab:

1. The required package versions are installed
2. The runtime is **automatically restarted** so the new versions can be loaded  
3. You may see the message **“Your session crashed for an unknown reason.”**  
   → This is expected and normal

After the restart, rerun the notebook and check the **version check cell** to confirm package versions are correct.

### Runtime menu notes
- **Runtime → Restart session**  
  Restarts Python but keeps installed packages and saved files

- **Runtime → Disconnect and delete runtime**  
  Resets Colab completely to its default environment (packages will need to be reinstalled)

In [ ]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])

In [ ]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)

---

## Imports

Run this cell once before starting the assignment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, suppress=True)


---

## Structure for this assignment

![Truss structure for Week 3](../Lectures/L3/assets/L1_Truss1.png)

### Geometry
- Width: $6\ \mathrm{m}$
- Height: $8\ \mathrm{m}$

We will model the structure as a 4-node truss with two diagonals (X-bracing).

### Node numbering
- Node 1: bottom left
- Node 2: top left
- Node 3: top right
- Node 4: bottom right

### Element directions to match the written assignment
The **stiffness** of an element does not depend on which node you call start/end, but the **transformation matrix does**. For the checks in this notebook, use the same start/end directions as the written assignment:

1. **Diagonal member**: start node = top left (2), end node = bottom right (4)
2. **Vertical member**: start node = bottom left (1), end node = top left (2)
3. **Horizontal member**: start node = top right (3), end node = top left (2)

Angle $\theta$ is measured as the counter-clockwise rotation from global $x$ (to the right) to the element’s local $x$ axis (from start to end node).


---

## Question 1 — Problem data setup + format checks (20 pts)

In this question you will define the problem data in a **strict format** that will be used by all later questions.

### Required variables (names must match exactly)

Create the following variables:

1. `W` (float): width in **mm** (must be `6000.0`)
2. `H` (float): height in **mm** (must be `8000.0`)
3. `nodes` (dict): maps node id (int) → NumPy array `[x, y]` of shape `(2,)` in **mm**
4. `E` (float): Young’s modulus in **MPa = N/mm^2**
5. `A_top_bot`, `A_vert`, `A_diag` (float): cross-sectional areas in **mm^2**
6. `elements` (dict): maps element name (str) → tuple `(i, j, A)` where:
   - `i` and `j` are **ints** (start node, end node)
   - `A` is a **float** (area in mm^2)

### Required element names
Your `elements` dict must include these keys exactly:

- `"bottom"`: (1, 4, A_top_bot)
- `"top"`: (2, 3, A_top_bot)
- `"left"`: (1, 2, A_vert)
- `"right"`: (4, 3, A_vert)
- `"diag_TL_BR"`: (2, 4, A_diag)
- `"diag_BL_TR"`: (1, 3, A_diag)

### Elements of interest (for direction checks)
We will refer to these **existing elements** by name later:

- diagonal: `elements["diag_TL_BR"]` (2 → 4)
- vertical: `elements["left"]` (1 → 2)
- horizontal: `elements["top"]` but **with direction** (3 → 2) for the transformation check

✅ Deliverable: Fill in the data cell so all tests pass.


In [ ]:
# ============================================================
# Q1 — YOUR WORK: define problem data in the required format
# Units: mm, N, MPa (= N/mm^2)
# ============================================================

# Geometry
W = None  # TODO
H = None  # TODO

# Nodes: dict[int -> np.ndarray shape (2,)]
nodes = None  # TODO

# Material
E = None  # TODO

# Areas (mm^2)
A_top_bot = None  # TODO
A_vert    = None  # TODO
A_diag    = None  # TODO

# Elements: dict[str -> (i:int, j:int, A:float)]
elements = None  # TODO

print("Nodes:")
if isinstance(nodes, dict):
    for i in [1, 2, 3, 4]:
        if i in nodes:
            print(f"  {i}: {nodes[i]}")

print("\nElements:")
if isinstance(elements, dict):
    for name, (i, j, A) in elements.items():
        print(f"  {name:10s}: {i} -> {j}, A = {A} mm^2")


In [ ]:
# ============================================================
# Q1 TESTS — Do Not Edit
# ============================================================

print("\n--- Q1 tests: data format + geometry checks ---")

# ------------------------------------------------------------
# 1) Basic scalar checks (units + expected values)
# ------------------------------------------------------------
assert isinstance(W, (int, float)), "W must be a number (mm)"
assert isinstance(H, (int, float)), "H must be a number (mm)"
W = float(W)
H = float(H)

np.testing.assert_allclose(W, 6000.0, atol=0, rtol=0)
np.testing.assert_allclose(H, 8000.0, atol=0, rtol=0)

assert isinstance(E, (int, float)), "E must be a number (MPa = N/mm^2)"
E = float(E)
assert E > 0, "E must be positive"

for name in ["A_top_bot", "A_vert", "A_diag"]:
    val = globals()[name]
    assert isinstance(val, (int, float)), f"{name} must be a number (mm^2)"
    assert float(val) > 0, f"{name} must be positive"

np.testing.assert_allclose(float(A_top_bot), 4000.0, atol=0, rtol=0)
np.testing.assert_allclose(float(A_vert),    6000.0, atol=0, rtol=0)
np.testing.assert_allclose(float(A_diag),    6000.0, atol=0, rtol=0)

# ------------------------------------------------------------
# 2) Nodes dictionary format + exact coordinates
# ------------------------------------------------------------
assert isinstance(nodes, dict), "nodes must be a dict"
assert set(nodes.keys()) == {1, 2, 3, 4}, "nodes must have keys {1,2,3,4}"

for nid, xy in nodes.items():
    assert isinstance(nid, int), "node keys must be ints"
    assert isinstance(xy, np.ndarray), f"nodes[{nid}] must be a NumPy array"
    assert xy.shape == (2,), f"nodes[{nid}] must have shape (2,)"
    assert np.issubdtype(xy.dtype, np.number), "node coordinates must be numeric"

# exact rectangle coordinates (mm)
np.testing.assert_allclose(nodes[1], [0.0, 0.0], atol=0, rtol=0)
np.testing.assert_allclose(nodes[2], [0.0, H],   atol=0, rtol=0)
np.testing.assert_allclose(nodes[3], [W,   H],   atol=0, rtol=0)
np.testing.assert_allclose(nodes[4], [W,   0.0], atol=0, rtol=0)

# ------------------------------------------------------------
# 3) Elements dictionary format + required connectivity
# ------------------------------------------------------------
assert isinstance(elements, dict), "elements must be a dict"

required_elems = {"bottom", "top", "left", "right", "diag_TL_BR", "diag_BL_TR"}
assert set(elements.keys()) == required_elems, (
    f"elements must have keys {sorted(required_elems)}"
)

for ename, tpl in elements.items():
    assert isinstance(ename, str), "element keys must be strings"
    assert isinstance(tpl, tuple) and len(tpl) == 3, (
        f"elements['{ename}'] must be a tuple (i, j, A)"
    )

    i, j, A = tpl
    assert isinstance(i, int) and isinstance(j, int), (
        f"elements['{ename}'] i and j must be ints"
    )
    assert i in nodes and j in nodes, (
        f"elements['{ename}'] references invalid node id(s)"
    )
    assert isinstance(A, (int, float)), f"elements['{ename}'] A must be numeric"
    assert float(A) > 0, f"elements['{ename}'] A must be positive"

# connectivity spot-checks (matches the written assignment convention)
assert elements["bottom"][0:2] == (1, 4)
assert elements["top"][0:2]    == (2, 3)
assert elements["left"][0:2]   == (1, 2)
assert elements["right"][0:2]  == (4, 3)
assert elements["diag_TL_BR"][0:2] == (2, 4)
assert elements["diag_BL_TR"][0:2] == (1, 3)

print("\n✅ Q1 passed: data format + geometry checks")


---

## Question 2 — Element geometry ($c$, $s$, $L$) (20 pts)

For an element oriented from node $i$ to node $j$, compute:

- $\Delta x = x_j - x_i$
- $\Delta y = y_j - y_i$
- $L = \sqrt{(\Delta x)^2 + (\Delta y)^2}$
- $c = \cos\theta = \Delta x / L$
- $s = \sin\theta = \Delta y / L$

Your function must return **Python floats** `(c, s, L)`.

✅ Deliverable: Implement `element_csL(xy_i, xy_j)` so the tests pass.


In [ ]:
# YOUR WORK

def element_csL(xy_i, xy_j):
    """
    Compute the element geometry and direction cosines for a 2D truss element oriented i → j.

    INPUTS
    -------
    xy_i : np.ndarray, shape (2,)
        Global coordinates of the START node i, [x_i, y_i] (mm).
    xy_j : np.ndarray, shape (2,)
        Global coordinates of the END node j,   [x_j, y_j] (mm).

    OUTPUTS
    --------
    (c, s, L) : tuple[float, float, float]
        c : float
            cos(theta) = Δx / L
        s : float
            sin(theta) = Δy / L
        L : float
            Element length = sqrt((Δx)^2 + (Δy)^2) (mm)

        theta is measured as the counter-clockwise rotation from global +x
        to the element's local +x axis (from node i to node j).
    """
    # TODO
    raise NotImplementedError


In [ ]:
# ============================================================
# Q2 TESTS — Do Not Edit
# ============================================================

print("\n--- Q2 tests: element geometry (c, s, L) ---")

# ------------------------------------------------------------
# Basic type + sanity check (3–4–5 triangle)
# ------------------------------------------------------------
xy_a = np.array([0.0, 0.0])
xy_b = np.array([3.0, 4.0])

c, s, L = element_csL(xy_a, xy_b)

assert isinstance(c, float), "c must be a Python float"
assert isinstance(s, float), "s must be a Python float"
assert isinstance(L, float), "L must be a Python float"

np.testing.assert_allclose([c, s, L], [0.6, 0.8, 5.0], atol=1e-12)

# ------------------------------------------------------------
# Problem-specific geometry checks (selected elements)
# ------------------------------------------------------------

# Diagonal: top-left (2) → bottom-right (4)
c, s, L = element_csL(nodes[2], nodes[4])
np.testing.assert_allclose(L, 10000.0, atol=1e-12)
np.testing.assert_allclose(c, 0.6, atol=1e-12)
np.testing.assert_allclose(s, -0.8, atol=1e-12)

# Vertical: bottom-left (1) → top-left (2)
c, s, L = element_csL(nodes[1], nodes[2])
np.testing.assert_allclose(L, 8000.0, atol=1e-12)
np.testing.assert_allclose(c, 0.0, atol=1e-12)
np.testing.assert_allclose(s, 1.0, atol=1e-12)

# Horizontal (direction check): top-right (3) → top-left (2)
c, s, L = element_csL(nodes[3], nodes[2])
np.testing.assert_allclose(L, 6000.0, atol=1e-12)
np.testing.assert_allclose(c, -1.0, atol=1e-12)
np.testing.assert_allclose(s, 0.0, atol=1e-12)

print("✅ Q2 passed: element geometry and direction cosines")


---

## Question 3 — Local truss stiffness $\mathbf{k}_\ell$ (20 pts)

For a 2D truss element in **local coordinates** (local $x$ axis along the bar), the 4×4 stiffness matrix is:

$$
\mathbf{k}_\ell = \frac{EA}{L}
\begin{bmatrix}
 1 & 0 & -1 & 0 \\
 0 & 0 &  0 & 0 \\
-1 & 0 &  1 & 0 \\
 0 & 0 &  0 & 0
\end{bmatrix}
$$

✅ Deliverable: Implement `k_local_truss(E, A, L)` so the tests pass.


In [ ]:
# YOUR WORK

def k_local_truss(E, A, L):
    """
    Build the 4×4 LOCAL stiffness matrix for a 2D truss (bar) element.

    INPUTS
    -------
    E : float
        Young's modulus (MPa = N/mm^2).
    A : float
        Cross-sectional area (mm^2).
    L : float
        Element length (mm).

    OUTPUT
    ------
    k_local : np.ndarray, shape (4, 4)
        Local stiffness matrix in the element's LOCAL coordinate system, with local x-axis
        aligned with the bar.

        DOF ordering (local):
            $\{u\}_\ell = [u_{i\ell}, v_{i\ell}, u_{j\ell}, v_{j\ell}]^T$

        For a truss element, only axial stiffness is present (transverse rows/cols are zero):

            $\mathbf{k}_\ell = (E A / L)\,\begin{bmatrix}
            1 & 0 & -1 & 0 \\
            0 & 0 &  0 & 0 \\
           -1 & 0 &  1 & 0 \\
            0 & 0 &  0 & 0
            \end{bmatrix}$
    """
    # TODO
    raise NotImplementedError


In [ ]:
# ============================================================
# TESTS — Do Not Edit
# ============================================================
print("\n--- Q3 tests: local stiffness ---")

# ------------------------------------------------------------
# 1) Basic sanity check (simple numbers)
# ------------------------------------------------------------
L_test = 1000.0
kL = k_local_truss(E, A_diag, L_test)

assert isinstance(kL, np.ndarray), "k_local_truss must return a NumPy array"
assert kL.shape == (4, 4), "k_local_truss must return a 4x4 matrix"

# symmetry
np.testing.assert_allclose(kL, kL.T, atol=0, rtol=0)

# expected axial pattern
k0 = E * A_diag / L_test
np.testing.assert_allclose(kL[0, 0],  k0)
np.testing.assert_allclose(kL[0, 2], -k0)
np.testing.assert_allclose(kL[2, 0], -k0)
np.testing.assert_allclose(kL[2, 2],  k0)

# transverse DOFs should be zero in local coords
np.testing.assert_allclose(kL[1, :], 0)
np.testing.assert_allclose(kL[3, :], 0)
np.testing.assert_allclose(kL[:, 1], 0)
np.testing.assert_allclose(kL[:, 3], 0)

# ------------------------------------------------------------
# 2) New case: scale check (different A and L)
# ------------------------------------------------------------
A_test = 2500.0
L_test2 = 2000.0
kL2 = k_local_truss(E, A_test, L_test2)

assert isinstance(kL2, np.ndarray)
assert kL2.shape == (4, 4)
np.testing.assert_allclose(kL2, kL2.T, atol=0, rtol=0)

k02 = E * A_test / L_test2
np.testing.assert_allclose(kL2[0, 0],  k02)
np.testing.assert_allclose(kL2[0, 2], -k02)
np.testing.assert_allclose(kL2[2, 0], -k02)
np.testing.assert_allclose(kL2[2, 2],  k02)

np.testing.assert_allclose(kL2[1, :], 0)
np.testing.assert_allclose(kL2[3, :], 0)
np.testing.assert_allclose(kL2[:, 1], 0)
np.testing.assert_allclose(kL2[:, 3], 0)

# ------------------------------------------------------------
# 3) Problem-specific check (uses your assignment area)
# ------------------------------------------------------------
# Use the diagonal area and the diagonal length from the problem geometry (2 -> 4)
_, _, L_diag = element_csL(nodes[2], nodes[4])
kL_diag = k_local_truss(E, A_diag, L_diag)

assert isinstance(kL_diag, np.ndarray)
assert kL_diag.shape == (4, 4)
np.testing.assert_allclose(kL_diag, kL_diag.T, atol=0, rtol=0)

k0_diag = E * A_diag / L_diag
np.testing.assert_allclose(kL_diag[0, 0],  k0_diag, atol=1e-12)
np.testing.assert_allclose(kL_diag[0, 2], -k0_diag, atol=1e-12)
np.testing.assert_allclose(kL_diag[2, 0], -k0_diag, atol=1e-12)
np.testing.assert_allclose(kL_diag[2, 2],  k0_diag, atol=1e-12)

np.testing.assert_allclose(kL_diag[1, :], 0, atol=0, rtol=0)
np.testing.assert_allclose(kL_diag[3, :], 0, atol=0, rtol=0)
np.testing.assert_allclose(kL_diag[:, 1], 0, atol=0, rtol=0)
np.testing.assert_allclose(kL_diag[:, 3], 0, atol=0, rtol=0)

print("✅ Q3 passed")


---

## Question 4 — Transformation matrix $\mathbf{T}$ (20 pts)

In this question you will construct the **coordinate transformation matrix** that maps
**global nodal displacements** to **local element displacements** for a 2D truss element.

### Required approach (do not skip)

Your implementation of `T_truss(xy_i, xy_j)` **must reuse** the function you wrote earlier:

- `element_csL(xy_i, xy_j)`

Specifically, inside `T_truss` you should:

1. Call
   $$
   (c, s, L) = \texttt{element\_csL}(xy_i, xy_j)
   $$
2. Use only the direction cosines $c=\cos\theta$ and $s=\sin\theta$  
   (the length $L$ is **not** needed in this step).
3. Assemble the transformation matrix exactly as defined below.

### Transformation matrix definition

The transformation matrix $\mathbf{T}$ is

$$
\mathbf{T}=
\begin{bmatrix}
 c & s & 0 & 0 \\
-s & c & 0 & 0 \\
 0 & 0 & c & s \\
 0 & 0 &-s & c
\end{bmatrix}
$$

This matrix maps displacements from the **global coordinate system**
to the **local coordinate system** of the element, where the local $x$-axis
points from the start node $i$ to the end node $j$ and
$\theta$ is measured counter-clockwise from the global $x$-axis.

✅ **Deliverable:** Implement `T_truss(xy_i, xy_j)` so all tests pass.


In [ ]:
# YOUR WORK

def T_truss(xy_i, xy_j):
    """
    Build the 4×4 transformation matrix for a 2D truss element oriented from node i → node j.

    INPUTS
    -------
    xy_i : np.ndarray, shape (2,)
        Global coordinates of the START node i, [x_i, y_i] (mm).
    xy_j : np.ndarray, shape (2,)
        Global coordinates of the END node j,   [x_j, y_j] (mm).

    OUTPUT
    ------
    T : np.ndarray, shape (4, 4)
        Transformation matrix mapping global nodal displacements to local nodal displacements:

            $\{u\}_\ell = \mathbf{T}\,\{u\}_g$

        where:
            $\{u\}_g = [u_{ix}, u_{iy}, u_{jx}, u_{jy}]^T$
            $\{u\}_\ell = [u_{i\ell}, v_{i\ell}, u_{j\ell}, v_{j\ell}]^T$

        The local x-axis points from node i to node j.
        The angle $\theta$ is measured counter-clockwise from global +x to the local +x axis.
    """
    # TODO
    raise NotImplementedError


In [ ]:
# ============================================================
# TESTS — Do Not Edit
# ============================================================
print("\n--- Q4 tests: transformation matrix ---")

# Diagonal TL -> BR: (2->4)
T = T_truss(nodes[2], nodes[4])
T_expected = np.array([
    [ 0.6, -0.8, 0.0,  0.0],
    [ 0.8,  0.6, 0.0,  0.0],
    [ 0.0,  0.0, 0.6, -0.8],
    [ 0.0,  0.0, 0.8,  0.6],
])
np.testing.assert_allclose(T, T_expected, atol=1e-12)

# Vertical BL -> TL: (1->2)
T = T_truss(nodes[1], nodes[2])
T_expected = np.array([
    [ 0.0, 1.0, 0.0, 0.0],
    [-1.0, 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 1.0],
    [ 0.0, 0.0,-1.0, 0.0],
])
np.testing.assert_allclose(T, T_expected, atol=1e-12)

# Horizontal TR -> TL: use nodes[3] -> nodes[2]
T = T_truss(nodes[3], nodes[2])
T_expected = np.array([
    [-1.0, 0.0, 0.0, 0.0],
    [-0.0,-1.0, 0.0, 0.0],
    [ 0.0, 0.0,-1.0, 0.0],
    [ 0.0, 0.0,-0.0,-1.0],
])
np.testing.assert_allclose(T, T_expected, atol=1e-12)

print("✅ Q4 passed")


## Question 5 — Global element stiffness $\mathbf{k}_g$ (20 pts)

In this question you will compute the **global stiffness matrix** for a single 2D truss element by combining the helper functions you have already written.

### Required workflow

Your implementation of `k_global_element(E, A, xy_i, xy_j)` must:

1. Call  
   $$
   (c, s, L) = \texttt{element\_csL}(xy_i, xy_j)
   $$
   to obtain the element length $L$.

2. Build the **local stiffness matrix**  
   $$
   \mathbf{k}_\ell = \texttt{k\_local\_truss}(E, A, L)
   $$

3. Build the **transformation matrix**  
   $$
   \mathbf{T} = \texttt{T\_truss}(xy_i, xy_j)
   $$

4. Compute the **global stiffness matrix**  
   $$
   \mathbf{k}_g = \mathbf{T}^T \mathbf{k}_\ell \mathbf{T}
   $$

### Output

Return $\mathbf{k}_g$ as a 4×4 NumPy array using the global DOF ordering  
$[u_{ix}, u_{iy}, u_{jx}, u_{jy}]$.

✅ **Deliverable:** Implement `k_global_element(E, A, xy_i, xy_j)` so the tests pass.


In [ ]:
# YOUR WORK

def k_global_element(E, A, xy_i, xy_j):
    """
    Compute the 4x4 GLOBAL stiffness matrix for a 2D truss element oriented i → j.

    Must be assembled using your earlier helper functions:

      1) (c, s, L) = element_csL(xy_i, xy_j)
      2) k_local   = k_local_truss(E, A, L)
      3) T         = T_truss(xy_i, xy_j)
      4) k_global  = T.T @ k_local @ T

    INPUTS
    -------
    E : float
        Young's modulus (MPa = N/mm^2).
    A : float
        Cross-sectional area (mm^2).
    xy_i : np.ndarray, shape (2,)
        Global coordinates of the START node i, [x_i, y_i] (mm).
    xy_j : np.ndarray, shape (2,)
        Global coordinates of the END node j,   [x_j, y_j] (mm).

    OUTPUT
    ------
    k_global : np.ndarray, shape (4, 4)
        Global stiffness matrix in GLOBAL coordinates using DOF ordering:
            $\{u\}_g = [u_{ix}, u_{iy}, u_{jx}, u_{jy}]^T$
    """
    # TODO:
    raise NotImplementedError



In [ ]:
# ============================================================
# Q5 TESTS — Do Not Edit
# ============================================================
print("\n--- Q5 tests: global element stiffness (problem geometry) ---")

# ------------------------------------------------------------
# A) Basic output + symmetry checks (generic)
# ------------------------------------------------------------
k = k_global_element(E, A_top_bot, nodes[3], nodes[2])

assert isinstance(k, np.ndarray), "k_global_element must return a NumPy array"
assert k.shape == (4, 4), "k_global_element must return a 4x4 matrix"
np.testing.assert_allclose(k, k.T, atol=1e-12, err_msg="k_g must be symmetric")

# ------------------------------------------------------------
# B) Problem-specific: horizontal member TR -> TL (3 -> 2)
#    Here c=-1, s=0, so the global matrix matches the local axial pattern.
# ------------------------------------------------------------
Lh = 6000.0
k0 = E * A_top_bot / Lh
k_expected = k0 * np.array(
    [
        [ 1.0, 0.0, -1.0, 0.0],
        [ 0.0, 0.0,  0.0, 0.0],
        [-1.0, 0.0,  1.0, 0.0],
        [ 0.0, 0.0,  0.0, 0.0],
    ],
    dtype=float
)

k_h = k_global_element(E, A_top_bot, nodes[3], nodes[2])
np.testing.assert_allclose(k_h, k_expected, atol=1e-10)
np.testing.assert_allclose(k_h, k_h.T, atol=1e-12)

# ------------------------------------------------------------
# C) Problem-specific: vertical member BL -> TL (1 -> 2)
#    Here c=0, s=1, so the stiffness acts in the global y direction.
# ------------------------------------------------------------
Lv = 8000.0
k0 = E * A_vert / Lv
k_expected = k0 * np.array(
    [
        [0.0,  0.0, 0.0,  0.0],
        [0.0,  1.0, 0.0, -1.0],
        [0.0,  0.0, 0.0,  0.0],
        [0.0, -1.0, 0.0,  1.0],
    ],
    dtype=float
)

k_v = k_global_element(E, A_vert, nodes[1], nodes[2])
np.testing.assert_allclose(k_v, k_expected, atol=1e-10)
np.testing.assert_allclose(k_v, k_v.T, atol=1e-12)

# ------------------------------------------------------------
# D) Problem-specific: diagonal TL -> BR (2 -> 4)
#    Here c=0.6, s=-0.8, L=10000. We check against the closed-form matrix.
# ------------------------------------------------------------
Ld = 10000.0
c = 0.6
s = -0.8
k0 = E * A_diag / Ld

k_expected = k0 * np.array(
    [
        [ c*c,  c*s, -c*c, -c*s],
        [ c*s,  s*s, -c*s, -s*s],
        [-c*c, -c*s,  c*c,  c*s],
        [-c*s, -s*s,  c*s,  s*s],
    ],
    dtype=float
)

k_d = k_global_element(E, A_diag, nodes[2], nodes[4])
np.testing.assert_allclose(k_d, k_expected, atol=1e-10)
np.testing.assert_allclose(k_d, k_d.T, atol=1e-12)

# ------------------------------------------------------------
# E) Sanity: element stiffness must have two rigid-body modes (rank = 2)
#    For a truss bar (4 DOF element), rank should be 2.
# ------------------------------------------------------------
for name, (i, j, A) in {
    "horizontal_TR_to_TL": (3, 2, A_top_bot),
    "vertical_BL_to_TL": (1, 2, A_vert),
    "diagonal_TL_to_BR": (2, 4, A_diag),
}.items():
    k_e = k_global_element(E, A, nodes[i], nodes[j])
    r = np.linalg.matrix_rank(k_e, tol=1e-8)
    assert r == 2, f"{name}: expected rank 2 for a 2D truss bar element, got {r}"

print("✅ Q5 passed: global element stiffness checks (problem geometry)")


In [ ]:
# ------------------------------------------------------------
# Loop through all elements and print global stiffness
# (geometry is computed internally via k_global_element)
# ------------------------------------------------------------
print("\n--- Element summary (using k_global_element only) ---\n")

for name, (i, j, A) in elements.items():
    xy_i = nodes[i]
    xy_j = nodes[j]

    # Global stiffness (internally calls element_csL, k_local_truss, T_truss)
    k_g = k_global_element(E, A, xy_i, xy_j)

    print(f"Element: {name}")
    print(f"  Nodes: {i} -> {j}")
    print(f"  Area A = {A:.2f} mm^2")

    print("  Global stiffness k_g:")
    print(k_g)

    # Symmetry check (required property of stiffness matrices)
    sym_err = np.linalg.norm(k_g - k_g.T)
    print(f"  Symmetry check ||k_g − k_g^T|| = {sym_err:.2e}")

    print("-" * 60)

print("✅ Finished printing all element data using k_global_element")


---

## Reflection (Required)

In 3–6 sentences:

- What part felt easiest?
- What part felt hardest?
- One bug you encountered and how you fixed it.
- One thing you still don’t understand.
- If you used collaboration or AI tools, briefly describe how.


YOUR TEXT ANSWER HERE: